In [1]:
import torch
from torch import optim, nn
from torch.autograd import Variable
# import pixiedust

In [2]:
x = torch.tensor([[0,0,0,0],[1,0,0,0],[0,1,0,0],[0,0,1,0],[1,1,0,0],[1,0,1,0],[0,1,1,0],[1,1,1,0],[0,0,0,1],[1,0,0,1],[0,1,0,1],[0,0,1,1],[1,1,0,1],[1,0,1,1],[0,1,1,1],[1,1,1,1.]])
x.requires_grad_()
target = torch.tensor([[0],[0],[0],[0],[0],[0],[0],[0],[1],[1],[1],[1],[1],[1],[1],[1.]])


#   Variables for performance metrics
epochs = 20
lr = 0.2
counter = 0

# Define 2 chained models
models = [
    nn.Sequential(
        nn.Linear(4, 3),
        nn.Tanh()
    ),
    nn.Sequential(
        nn.Linear(3, 1),
        nn.Sigmoid()
    )
]

# Create optimisers for each segment and link to their segment
optimizers = [
    optim.SGD(params=model.parameters(),lr=lr)
    for model in models
]

In [3]:
def train():
    # Training Logic
    for iter in range(epochs):

        # 1) erase previous gradients (if they exist)
        for opt in optimizers:
            opt.zero_grad()

        # 2) make a prediction
        a  = models[0](x)

        # 3) send the activation signal to the next model
        remote_a = a.detach()
        # re-enable autograd here
        remote_a.requires_grad_()

        pred =  models[1](remote_a)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()
        
        # 5) Backprop gradient to model behind
        grad_a = remote_a.grad.clone()
        
        # 5) Go backward from grad_a 
        print(models[0][0].weight.grad)
        a.backward(grad_a)
        print(models[0][0].weight.grad)
        print(models[0][0].weight)


        # 5) change the weights
        for opt in optimizers:
            opt.step()

        # 6) print our progress
        # Do not use .data
        print(loss.detach())
        
train()

None
tensor([[ 0.0558,  0.0670,  0.0665,  0.3947],
        [-0.0059, -0.0040, -0.0066, -0.0304],
        [ 0.0059,  0.0073,  0.0082,  0.0533]])
Parameter containing:
tensor([[-0.0938,  0.3042,  0.3276, -0.1152],
        [ 0.1320, -0.3280,  0.2153, -0.4838],
        [ 0.4335,  0.2187, -0.1787,  0.0512]], requires_grad=True)
tensor(4.0880)
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[ 0.0337,  0.0499,  0.0604,  0.4350],
        [ 0.0449,  0.0225,  0.0570,  0.3011],
        [-0.0031, -0.0038, -0.0053, -0.0451]])
Parameter containing:
tensor([[-0.1050,  0.2908,  0.3143, -0.1941],
        [ 0.1332, -0.3272,  0.2166, -0.4778],
        [ 0.4323,  0.2172, -0.1803,  0.0405]], requires_grad=True)
tensor(3.8292)
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[ 0.0014,  0.0238,  0.0525,  0.5125],
        [ 0.0506, -0.0085,  0.0864,  0.5650],
        [-0.0021, -0.0024, -0.0082, -0.0962]])
Parameter containing:
tensor